In [2]:
from pathlib import Path

# path to wherever Jupyter is launched from
project_root = Path.cwd()  

# now build a path to .qasm files
qasm_dir   = project_root / "baseline"
qasm_file_paths = sorted(qasm_dir.glob("*.qasm"))


if not qasm_file_paths:
    raise FileNotFoundError(f"No .qasm files found in {qasm_dir}")


In [3]:
from bloqade import qasm2
from bloqade.qasm2.parse.lowering import QASM2
from bloqade.qasm2.passes import QASM2Py

# parse & lower each one
programs = {}
for path in qasm_file_paths:
    prog = QASM2(qasm2.main).loadfile(file=path)

    """
    reinterpret into Bloqade's parallelization-friendly intermediate representation. 
    Similar behaviour could have been obtained by just using qasm2.extended above
    """
    QASM2Py(prog.dialects)(prog)
    prog = prog.similar(qasm2.extended)

    programs[Path(path).stem] = prog
    print(f"→ {path} parsed & lowered: {prog}")

# `programs` now holds each file’s lowered IR under its filename-stem.


→ /Users/radek/Documents/universidad/hackthon/ETH_Qhack/assets/baseline/0.1.qasm parsed & lowered: Method("0.1")
→ /Users/radek/Documents/universidad/hackthon/ETH_Qhack/assets/baseline/0.2.qasm parsed & lowered: Method("0.2")
→ /Users/radek/Documents/universidad/hackthon/ETH_Qhack/assets/baseline/0.4.qasm parsed & lowered: Method("0.4")
→ /Users/radek/Documents/universidad/hackthon/ETH_Qhack/assets/baseline/1.qasm parsed & lowered: Method("1")
→ /Users/radek/Documents/universidad/hackthon/ETH_Qhack/assets/baseline/2.qasm parsed & lowered: Method("2")
→ /Users/radek/Documents/universidad/hackthon/ETH_Qhack/assets/baseline/3.qasm parsed & lowered: Method("3")
→ /Users/radek/Documents/universidad/hackthon/ETH_Qhack/assets/baseline/4.qasm parsed & lowered: Method("4")
→ /Users/radek/Documents/universidad/hackthon/ETH_Qhack/assets/baseline/qft.qasm parsed & lowered: Method("qft")


In [4]:
from bloqade.qasm2.emit import QASM2 # the QASM2 target
from bloqade.qasm2.parse import pprint # the QASM2 pretty printer

target = QASM2(allow_parallel=True)

In [5]:
program_ast = target.emit(programs["qft"])
pprint(program_ast)

KIRIN {func,lowering.call,lowering.func,py.ilist,qasm2.core,qasm2.expr,qasm2.indexing,qasm2.parallel,qasm2.uop,scf};
include "qelib1.inc";
qreg var_0[3];
creg var_1[3];
h var_0[2];
crz (1.5707963267948966) var_0[1], var_0[2];
crz (0.7853981633974483) var_0[0], var_0[2];
h var_0[1];
crz (1.5707963267948966) var_0[0], var_0[1];
h var_0[0];


In [18]:
from kirin import ir
from qiskit import QuantumCircuit

from bloqade.qasm2.rewrite.native_gates import RydbergGateSetRewriteRule
from kirin import ir
from kirin.rewrite import Walk
from bloqade.qasm2.passes import UOpToParallel, QASM2Fold


@ir.dialect_group(qasm2.extended)
def extended_opt(self):
    native_rewrite = Walk(RydbergGateSetRewriteRule(self)) # use Kirin's functionality to walk code line by line while applying neutral-atom gate decomposition as defined in Bloqade
    parallelize_pass = UOpToParallel(self) # review the code and apply parallelization using a heuristic
    agg_fold = QASM2Fold(self) # supports parallelization by unfolding loops to search for parallelization opportunities

    # here we define our new compiler pass
    def run_pass(
        kernel: ir.Method,
        *,
        fold: bool = True,
        typeinfer: bool = True,
        parallelize: bool = False,
    ):
        assert qasm2.extended.run_pass is not None
        qasm2.extended.run_pass(kernel, fold=fold, typeinfer=typeinfer) # apply the original run_pass to the lowered kernel
        native_rewrite.rewrite(kernel.code) # decompose all gates in the circuit to neutral atom gate set

        # here goes our parallelization optimizer; the order of the commands here matters!
        if parallelize:
            agg_fold.fixpoint(kernel)
            parallelize_pass(kernel)

    return run_pass


In [19]:
# helper to go from Method → Qiskit
def method_to_qiskit(method: ir.Method, *,parallelize: bool = True) -> QuantumCircuit:
    # run extended pass in case forgot
    extended_opt.run_pass(method, parallelize=parallelize)
    # emit OpenQASM2 text
    qasm = QASM2().emit_str(method)
    # parse into a Qiskit circuit
    return QuantumCircuit.from_qasm_str(qasm)

In [20]:
qc = method_to_qiskit(program_ast)
#print(qc.draw(output="text"))
fig = qc.draw(output="mpl", fold=120, scale=0.7)
display(fig)   # in a Jupyter notebook

AttributeError: 'MainProgram' object has no attribute 'verify'